In [1]:
import torch
import torchvision
from torch import nn
from torchinfo import summary
from torch.utils.data import DataLoader
from module.data_processes import get_data
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

In [2]:
writer=SummaryWriter(log_dir='runs/data_10_perc-effnetb0-5_epochs-lr_0_001')

In [3]:
train_dir, test_dir=get_data(
    data_path='data/pizza_steak_sushi',
    data_url='https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip',
    unzip=True,
)

# auto creation of transformations
weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_transforms=weights.transforms()

In [4]:
BATCH_SIZE=32
DEVICE='cpu'

In [5]:
trainDataset=datasets.ImageFolder(
    root=train_dir,
    transform=auto_transforms,
) # creating training dataset

train_dataloader=DataLoader(
    trainDataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
) # creating training dataloader

# getting the name and indexer of classes:
class_name=trainDataset.classes
class_dict=trainDataset.class_to_idx

# setting up the testing dataset
testDataset=datasets.ImageFolder(
    root=test_dir,
    transform=auto_transforms,
) # creating testing dataset

# creating the testing dataloader
test_dataloader=DataLoader(
    testDataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
) # creating testing dataloader

In [6]:
model=torchvision.models.efficientnet_b0(weights=weights).to(DEVICE)

In [7]:
# # getting a summary of our model with torchinfo.summary()
# summary(
#     model, 
#     input_size=(1,3,224,224),
#     col_names=['input_size', 'output_size', 'num_params', 'trainable'],
#     col_width=15,
#     row_settings=['var_names']
# )

In [8]:
for param in model.features.parameters():
    param.requires_grad=False

# Update the classifier head of our model to suit the problem
model.classifier=nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(
        in_features=1280,
        out_features=len(class_name)
    )
).to(DEVICE)

In [9]:
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
def step(
    dataLoader,
    device='cpu',
    infer=False,
):
    if infer:
        model.eval()
    else:
        model.train()

    currentLoss, currentMetric=0, 0
    for batch, (X,y) in enumerate(dataLoader):
        X,y=X.to(device), y.to(device)
        y_pred=model(X)
        loss=loss_fn(y_pred, y)
        currentLoss += loss.item()

        if not infer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        y_pred_output=torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        currentMetric+=(y_pred_output==y).sum().item()/len(y_pred)

    currentLoss=currentLoss/len(dataLoader)
    currentMetric=currentMetric/len(dataLoader)
    return currentLoss, currentMetric

In [11]:
epochs=5
for epoch in tqdm(range(epochs), total=epochs):
    train_loss, train_acc=step(
        train_dataloader,
    )

    test_loss, test_acc=step(
        test_dataloader,
        infer=True,
    )
    # print(f"Epoch: {epoch} | train loss: {train_loss} | train_acc: {train_acc}")
    # print(f"test loss: {test_loss} | test_acc: {test_acc}")

    writer.add_scalars(
        main_tag='Loss',
        tag_scalar_dict={
            'train_loss':train_loss,
            'test_loss':test_loss,
        },
        global_step=epoch
    )

    writer.add_scalars(
        main_tag='Accuracy',
        tag_scalar_dict={
            'train_acc':train_acc,
            'test_acc':test_acc,
        },
        global_step=epoch
    )

    writer.add_graph(
        model=model,
        input_to_model=torch.randn(32,3,224,224).to(DEVICE)
    )
writer.close()

  0%|          | 0/5 [00:00<?, ?it/s]

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs